In [128]:
import os
from collections import defaultdict
import numpy as np
import prody
import pandas as pd

# import nglview
# import matplotlib.pyplot as plt

## List of PDBs for Testing

## Calculate Solvent Accessibility and Relative Solvent Accessibility

In [127]:
# http://prowl.rockefeller.edu/aainfo/volume.htm
AA_SA_VOL = pd.DataFrame({'resn':         ['ALA', 'ARG', 'ASP', 'ASN', 'CYS', 'GLU', 'GLN', 'GLY', 'HIS', 'ILE', 
                                           'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL'],
                          'surface_area': [ 115.,  225.,  150.,  160.,  135.,  190.,  180.,   75.,  195.,  175., 
                                            170.,  200.,  185.,  210.,  145.,  115.,  140.,  255.,  230.,  155.],
                          'volume':       [ 88.6, 173.6, 111.1, 114.1, 108.5, 138.4, 143.8,  60.1, 153.2, 166.7, 
                                           166.7, 168.6, 162.9, 189.9, 112.7,  89.0, 116.1, 227.8, 193.6, 140.0]})


def run_dssp(pdb_id):
    """
    Run DSSP on a PDB file and return the resulting AtomGroup
    
    :param pdb_id: String containing PDB ID
    """
    # TODO: how to silence the DSSP functions completely
    
    file_ext_list = ['pdb', 'pdb.gz', 'dssp']
    gzip_file = pdb_id + '.pdb.gz'
    dssp_file = pdb_id + '.dssp'

    ag = prody.parsePDB(pdb_id, stderr=False)
    prody.execDSSP(gzip_file, stderr=False)
    prody.parseDSSP(dssp_file, ag)

    # File cleanup -- maybe there is a better way to handle this?
    for ext in file_ext_list:
        filename = '.'.join([pdb_id, ext])
        if os.path.isfile(filename):
            os.remove(filename)
            
    return ag


def gather_residue_dssp_data(atom_group):
    """
    Return DSSP data on a per-residue level and return as a dataframe
    
    :param atom_group: ProDyn AtomGroup instance
    """

    attr_list = ['dssp_acc', 'dssp_kappa', 'dssp_alpha', 'dssp_phi', 'dssp_psi']
    dssp_dict = defaultdict(list)

    for res in ag.iterResidues():
        pdb_resi = res.getResindex()
        dssp_resi = int(np.mean(res.getData('dssp_resnum')))
        
        if dssp_resi != 0:
            dssp_dict['resi'].append(pdb_resi)
            dssp_dict['resn'].append(res.getResname())
            dssp_dict['dssp_resi'].append(dssp_resi)
            
            for attr in attr_list:
                atom_data = np.array(res.getData(attr))
                assert np.allclose(atom_data.mean(), atom_data.min())
                dssp_dict[attr].append(atom_data.mean())

    dssp_df = pd.DataFrame(dssp_dict)
    return dssp_df


def calc_relative_solvent_sa(dssp_df, sa_df=AA_SA_VOL[['resn', 'surface_area']], 
                             resn_col='resn', resi_col='resi', sa_col='surface_area', rel_sa_col='dssp_acc_rel'):
    """
    Calculate percentage surface area
    
    :param dssp_df: DataFrame with aggregated DSSP results -- output from `gather_residue_dssp_data`
    :param sa_df: DataFrame with per-residue surface accessible area measurements default is AA_SA_VOL
    :param resn_col: String that holds the column name for residue names
    :param resi_col: String thad holds the column name for residue numbers
    :param sa_col: String that holds the column name for surface area
    :param rel_sa_col: String that holds the column name 
    """
    rel_sa_df = pd.merge(dssp_df, sa_df, on=resn_col, how='left').sort_values(resi_col)
    rel_sa_df[rel_sa_col] = rel_sa_df['dssp_acc'] / rel_sa_df[sa_col]
    rel_sa_df.drop(sa_col, axis=1, inplace=True)
    return rel_sa_df

def 
    
    
pdb_ids = "5r7y, 5r7z, 5r80, 5r81, 5r82, 5r83, 5r84, 5re4, 5re5, 5re6, 5re7, 5re8, 5re9, 5rea, 5reb, 5rec, 5red, 5ree, 5ref, 5reg, 5reh, 5rei, 5rej, 5rek, 5rel, 5rem, 5ren, 5reo, 5rep, 5rer, 5res, 5ret, 5reu, 5rev, 5rew, 5rex, 5rey, 5rez, 5rf0, 5rf1, 5rf2, 5rf3, 5rf4, 5rf5, 5rf6, 5rf7, 5rf8, 5rf9, 5rfa, 5rfb, 5rfc, 5rfd, 5rfe, 5rff, 5rfg, 5rfh, 5rfi, 5rfj, 5rfk, 5rfl, 5rfm, 5rfn, 5rfo, 5rfp, 5rfq, 5rfr, 5rfs, 5rft, 5rfu, 5rfv, 5rfw, 5rfx, 5rfy, 5rfz, 5rg0, 6lu7, 6m03, 6w63, 6y2e, 6y2f, 6y2g, 6y84, 6yb7"
pdb_ids = pdb_ids.split(", ")


pdb_id = pdb_ids[0]
atom_group = run_dssp(pdb_id)
dssp_df = gather_residue_dssp_data(atom_group)
calc_relative_solvent_sa(dssp_df)

@> Connecting wwPDB FTP server RCSB PDB (USA).
@> 5r7y downloaded (5r7y.pdb.gz)
@> PDB download via FTP completed (1 downloaded, 0 failed).
@> 2698 atoms and 1 coordinate set(s) were parsed in 0.04s.


,resi,resn,dssp_resi,dssp_acc,dssp_kappa,dssp_alpha,dssp_phi,dssp_psi,dssp_acc_rel
0,0,SER,1,157.0,360.0,360.0,360.0,170.9,1.365217
1,1,GLY,2,48.0,360.0,-130.7,98.3,147.1,0.640000
2,2,PHE,3,22.0,27.7,178.9,-137.9,121.1,0.104762
3,3,ARG,4,193.0,42.4,-103.6,-126.6,151.5,0.857778
4,4,LYS,5,84.0,57.1,-169.2,-71.2,110.9,0.420000
...,...,...,...,...,...,...,...,...,...
299,299,CYS,300,50.0,121.0,36.6,-78.1,-14.7,0.370370
300,300,SER,301,82.0,106.8,-125.6,-106.5,-29.5,0.713043
301,301,GLY,302,50.0,32.0,-172.7,89.3,37.9,0.666667
302,302,VAL,303,90.0,360.0,360.0,-56.1,146.0,0.580645


In [115]:
dssp_df.columns

Index(['resi', 'resn', 'dssp_resi', 'dssp_acc', 'dssp_kappa', 'dssp_alpha',
       'dssp_phi', 'dssp_psi'],
      dtype='object')

In [122]:
dssp_df[['resi', 'resn', 'dssp_acc']].set_index('resn').div(sa_df.set_index('resn').rename(columns={'surface_area': 'dssp_acc'}))

,dssp_acc,resi
resn,,
ALA,0.252174,NaN
ALA,0.165217,NaN
ALA,0.486957,NaN
ALA,0.052174,NaN
ALA,0.000000,NaN
...,...,...
VAL,0.135484,NaN
VAL,0.064516,NaN
VAL,0.000000,NaN


In [123]:
dssp_df.div?

Signature: dssp_df.div(other, axis='columns', level=None, fill_value=None)
Docstring:
Get Floating division of dataframe and other, element-wise (binary operator `truediv`).

Equivalent to ``dataframe / other``, but with support to substitute a fill_value
for missing data in one of the inputs. With reverse version, `rtruediv`.

Among flexible wrappers (`add`, `sub`, `mul`, `div`, `mod`, `pow`) to
arithmetic operators: `+`, `-`, `*`, `/`, `//`, `%`, `**`.

Parameters
----------
other : scalar, sequence, Series, or DataFrame
    Any single or multiple element data structure, or list-like object.
axis : {0 or 'index', 1 or 'columns'}
    Whether to compare by the index (0 or 'index') or columns
    (1 or 'columns'). For Series input, axis to match Series index on.
level : int or label
    Broadcast across a level, matching Index values on the
    passed MultiIndex level.
fill_value : float or None, default None
    Fill existing missing (NaN) values, and any new element needed for
    suc

,resi,resn,dssp_resi,dssp_acc,dssp_kappa,dssp_alpha,dssp_phi,dssp_psi
0,0,SER,1,157.0,360.0,360.0,360.0,170.9
1,1,GLY,2,48.0,360.0,-130.7,98.3,147.1
2,2,PHE,3,22.0,27.7,178.9,-137.9,121.1
3,3,ARG,4,193.0,42.4,-103.6,-126.6,151.5
4,4,LYS,5,84.0,57.1,-169.2,-71.2,110.9
...,...,...,...,...,...,...,...,...
299,299,CYS,300,50.0,121.0,36.6,-78.1,-14.7
300,300,SER,301,82.0,106.8,-125.6,-106.5,-29.5
301,301,GLY,302,50.0,32.0,-172.7,89.3,37.9
302,302,VAL,303,90.0,360.0,360.0,-56.1,146.0


In [58]:
res.getData('dssp_resnum')

array([0])

In [67]:
res.getData('dssp_acc')

array([0.])

In [57]:
for res in ag.iterResidues():
    print(res.getResindex(), res.getResname())

0 SER
1 GLY
2 PHE
3 ARG
4 LYS
5 MET
6 ALA
7 PHE
8 PRO
9 SER
10 GLY
11 LYS
12 VAL
13 GLU
14 GLY
15 CYS
16 MET
17 VAL
18 GLN
19 VAL
20 THR
21 CYS
22 GLY
23 THR
24 THR
25 THR
26 LEU
27 ASN
28 GLY
29 LEU
30 TRP
31 LEU
32 ASP
33 ASP
34 VAL
35 VAL
36 TYR
37 CYS
38 PRO
39 ARG
40 HIS
41 VAL
42 ILE
43 CYS
44 THR
45 SER
46 GLU
47 ASP
48 MET
49 LEU
50 ASN
51 PRO
52 ASN
53 TYR
54 GLU
55 ASP
56 LEU
57 LEU
58 ILE
59 ARG
60 LYS
61 SER
62 ASN
63 HIS
64 ASN
65 PHE
66 LEU
67 VAL
68 GLN
69 ALA
70 GLY
71 ASN
72 VAL
73 GLN
74 LEU
75 ARG
76 VAL
77 ILE
78 GLY
79 HIS
80 SER
81 MET
82 GLN
83 ASN
84 CYS
85 VAL
86 LEU
87 LYS
88 LEU
89 LYS
90 VAL
91 ASP
92 THR
93 ALA
94 ASN
95 PRO
96 LYS
97 THR
98 PRO
99 LYS
100 TYR
101 LYS
102 PHE
103 VAL
104 ARG
105 ILE
106 GLN
107 PRO
108 GLY
109 GLN
110 THR
111 PHE
112 SER
113 VAL
114 LEU
115 ALA
116 CYS
117 TYR
118 ASN
119 GLY
120 SER
121 PRO
122 SER
123 GLY
124 VAL
125 TYR
126 GLN
127 CYS
128 ALA
129 MET
130 ARG
131 PRO
132 ASN
133 PHE
134 THR
135 ILE
136 LYS
137 GLY
138 SE

In [56]:
res.getResname()

'SER'

In [52]:
dssp_df.groupby('dssp_resnum').agg(['min', 'max'])

dssp_acc        dssp_kappa        dssp_alpha        dssp_phi  \
                 min    max        min    max        min    max      min   
dssp_resnum                                                                
0                0.0    0.0        0.0    0.0        0.0    0.0      0.0   
1              157.0  157.0      360.0  360.0      360.0  360.0    360.0   
2               48.0   48.0      360.0  360.0     -130.7 -130.7     98.3   
3               22.0   22.0       27.7   27.7      178.9  178.9   -137.9   
4              193.0  193.0       42.4   42.4     -103.6 -103.6   -126.6   
...              ...    ...        ...    ...        ...    ...      ...   
300             50.0   50.0      121.0  121.0       36.6   36.6    -78.1   
301             82.0   82.0      106.8  106.8     -125.6 -125.6   -106.5   
302             50.0   50.0       32.0   32.0     -172.7 -172.7     89.3   
303             90.0   90.0      360.0  360.0      360.0  360.0    -56.1   
304            149.0  149.0      360.0  360.0      360.0  360.0   -124.6   

                   dssp_psi         
               max      min    max  
dssp_resnum                         
0              0.0      0.0    0.0  
1            360.0    170.9  170.9  
2             98.3    147.1  147.1  
3           -137.9    121.1  121.1  
4           -126.6    151.5  151.5  
...            ...      ...    ...  
300          -78.1    -14.7  -14.7  
301         -106.5    -29.5  -29.5  
302           89.3     37.9   37.9  
303          -56.1    146.0  146.0  
304         -124.6    360.0  360.0  

[305 rows x 10 columns]

array([170.9, 170.9, 170.9, ...,   0. ,   0. ,   0. ])